## Notebook to create an anndata object with the demuxlet identified samples mappings per sample pool via papermill

In [ ]:
!date

#### import libraries and set notebook variables

In [ ]:
import os
import papermill as pm
import concurrent.futures
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# naming
cohort = 'aging'

# directories
home_dir = f'/labshare/raph/notebooks/expression/adrd_neuro/{cohort}'

# base notebook to be run per tuple iteration
base_notebook = f'{home_dir}/glmmtmb_diffexp.ipynb'

# output path for the generated notebooks
out_nb_dir = f'{home_dir}/pm_gend_nbs'

# setup parameters to iterate
brain_regions = ['Entorhinal cortex', 'Putamen', 'Subventricular zone', 
                 'Middle temporal gyrus']
cell_types = ['Oligodendrocyte-1', 'SPN D1', 'SPN D2', 'Oligodendrocyte-2', 
              'Astrocyte', 'ExN CUX2 LAMP5', 'InN ADARB2 VIP', 'ExN FEZF2', 
              'OPC', 'ExN RORB THEMIS', 'InN LHX6 PVALB', 'Radial Glia', 
              'Microglia', 'InN ADARB2 LAMP5', 'ExN CUX2 ADARB2', 
              'InN LHX6 SST', 'SPN D1-2', 'Endothelial', 'ExN RORB', 
              'ExN LAMP5', 'Astrocyte-GFAP-Hi', 'SPN D2-2', 'ExN THEMIS']
groupings = {'Brain_region': brain_regions, 'new_anno': cell_types}

#### utility functions

In [ ]:
def run_pm_notebook(base_notebook: str, out_notebook: str, params: dict) -> str:
    ret_val = f'notebook: {out_notebook}\nparams: {params}'
    pm.execute_notebook(input_path=base_notebook, output_path=out_notebook, 
                        parameters=params, progress_bar=False)
    return ret_val

#### check the notebook template

In [ ]:
pm.inspect_notebook(base_notebook)

#### iterate over the list running the notebook per sample pool

In [ ]:
%%time
# make sure the notebook output dir exists
os.makedirs(out_nb_dir, exist_ok=True)

futures = []
results = []
with concurrent.futures.ProcessPoolExecutor() as ppe:
    for g_type, groups in groupings.items():
        for grouping in groups:
            param_dict = {'region_celltype': grouping, 'obs_type': g_type, 'testing': True}
            out_notebook = f'{out_nb_dir}/{grouping.replace(" ", "_")}.glmmtmb_diffexp.ipynb'
            futures.append(ppe.submit(run_pm_notebook, base_notebook, out_notebook, param_dict))

In [ ]:
for future in concurrent.futures.as_completed(futures):
    results.append(future.result())

In [ ]:
print(results)